In [ ]:
# ============================================================
# SETUP AND DEPENDENCIES
# ============================================================

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel, Matern
from sklearn.ensemble import RandomForestRegressor
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)

print("Week 9 ML Models and Strategy")
print("="*60)
print("CONTEXT: Recovery from Week 8 disaster")
print("Week 8: Changed F5 x1 from 1.0 to 0.855 -> crashed to 4416")
print("Week 9: Revert x1 to 1.0, single-variable isolation")
print("="*60)

# ============================================================
# DATA LOADING (WEEKS 1-8)
# ============================================================

# All historical data through Week 8
data_w8 = {
    'F1': {
        'X': np.array([[0.10,0.10],[0.12,0.08],[0.21,0.11],[0.14,0.14],
                       [0.08,0.08],[0.45,0.45],[0.48,0.48],[0.405,0.428]]),
        'y': np.array([0.0,0.0,0.0,0.0,0.0,0.0128,0.000008,0.462531]),
        'dim': 2
    },
    'F2': {
        'X': np.array([[0.10,0.10],[0.12,0.08],[0.21,0.11],[0.14,0.14],
                       [0.08,0.08],[0.111,0.100],[0.11,0.10],[0.111,0.100]]),
        'y': np.array([0.0892,0.0705,0.0295,0.0150,0.0463,0.1300,0.0468,
                       -0.0246]),
        'dim': 2
    },
    'F3': {
        'X': np.array([[0.80,0.80,0.80],[0.95,0.95,0.95],[0.98,0.99,0.87],
                       [0.948885,0.965632,0.808397],[1.01,1.01,0.82],
                       [0.928,0.832,0.004],[0.99,0.99,0.99],
                       [0.944,0.965,0.807]]),
        'y': np.array([-0.1055,-0.0919,-0.0856,-0.0786,-1.1543,-0.1161,
                       -0.427251,-0.088593]),
        'dim': 3
    },
    'F4': {
        'X': np.array([[0.5,0.5,0.5,0.5],[0.3,0.3,0.3,0.3],
                       [0.44,0.29,0.35,1.25],[0.51,0.60,0.57,0.01],
                       [0.66,0.30,0.30,0.36],[0.2,0.2,0.95,0.4],
                       [0.65,0.65,0.65,0.65],[0.498,0.502,0.500,0.500]]),
        'y': np.array([-3.986,-4.306,-30.129,-12.492,-7.262,-19.009,
                       -15.158,-3.9857]),
        'dim': 4
    },
    'F5': {
        'X': np.array([[0.30,0.30,0.30,0.30],[0.28,0.32,0.30,0.29],
                       [0.344822,0.264687,0.374156,0.203902],
                       [0.196828,0.320017,0.300,0.289958],
                       [0.99,0.90,0.98,0.93],[0.985,0.905,0.975,0.925],
                       [1.0,0.853,1.0,0.977],[0.855,0.852,1.000,0.979]]),
        'y': np.array([136.85,137.29,131.78,140.74,5549.45,5398.58,
                       6158.08,4415.99]),
        'dim': 4
    },
    'F6': {
        'X': np.array([[0.75,0.75,0.75,0.75,0.75],[0.3,0.3,0.3,0.3,0.3],
                       [0.49,0.02,0.45,0.40,0.32],[0.69,0.001,0.04,0.001,0.001],
                       [0.26,0.18,0.50,0.48,0.41],[0.1,0.1,0.7,0.7,0.6],
                       [0.15,0.15,0.50,0.50,0.70],[0.258,0.178,0.501,0.482,0.412]]),
        'y': np.array([-1.521,-1.139,-1.123,-2.067,-1.092,-1.231,-1.5517,
                       -1.064064]),
        'dim': 5
    },
    'F7': {
        'X': np.array([[1.0,1.0,1.0,1.0,1.0,1.0],[0.2,0.2,0.2,0.2,0.2,0.2],
                       [0.21,0.19,0.21,0.19,0.17,0.19],
                       [0.08,0.32,0.15,0.28,0.41,0.27],
                       [0.05,0.50,0.25,0.20,0.15,0.85],
                       [0.06,0.48,0.25,0.20,0.40,0.75],
                       [0.038,0.462,0.239,0.171,0.378,0.734],
                       [0.039,0.463,0.240,0.172,0.379,0.742]]),
        'y': np.array([0.000034,0.408,0.347,0.568,0.836,1.435,1.478289,
                       1.463533]),
        'dim': 6
    },
    'F8': {
        'X': np.array([[0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1],
                       [0.12,0.09,0.11,0.10,0.08,0.13,0.11,0.09],
                       [0.29,0.25,0.02,0.29,0.14,0.22,0.25,0.30],
                       [1.0,0.001,1.0,0.001,0.001,1.0,1.0,0.001],
                       [0.05,0.25,0.25,0.25,0.25,0.25,0.05,0.05],
                       [0.18,0.15,0.20,0.15,0.25,0.15,0.15,0.18],
                       [0.177,0.194,0.170,0.194,0.294,0.143,0.109,0.208],
                       [0.179,0.196,0.172,0.196,0.292,0.145,0.111,0.210]]),
        'y': np.array([9.542,9.554,9.548,4.180,9.643,9.676,9.692075,
                       9.691885]),
        'dim': 8
    }
}

print("\nData loaded: 8 weeks of history for all 8 functions\n")

# ============================================================
# F1 (2D): GAUSSIAN PROCESS - MINOR REFINEMENT
# ============================================================

print("="*60)
print("F1 (2D): GAUSSIAN PROCESS - PEAK REFINEMENT")
print("="*60)

X_f1 = data_w8['F1']['X']
y_f1 = data_w8['F1']['y']

# Remove zeros for GP training
nonzero_mask = y_f1 > 0
X_f1_nz = X_f1[nonzero_mask]
y_f1_nz = y_f1[nonzero_mask]

print(f"\nTraining on non-zero data: {len(y_f1_nz)} points")
print(f"Output range: {y_f1_nz.min():.6f} to {y_f1_nz.max():.6f}")

# GP with Matern kernel
kernel_f1 = ConstantKernel(1.0) * Matern(
    length_scale=np.ones(2),
    length_scale_bounds=(1e-3, 10),
    nu=2.5
)

gp_f1 = GaussianProcessRegressor(
    kernel=kernel_f1,
    n_restarts_optimizer=10,
    alpha=1e-6,
    normalize_y=True
)

gp_f1.fit(X_f1_nz, y_f1_nz)

print("\nHistorical progression:")
print("  W6: [0.45, 0.45] -> 0.0128")
print("  W7: [0.48, 0.48] -> 0.000008 (overshot)")
print("  W8: [0.405, 0.428] -> 0.462531 (BEST)")

print("\nStrategy: Micro-adjustment from W8 peak")

# Test candidates near W8 peak
candidates_f1 = {
    'W8 best': [0.405, 0.428],
    'Slight lower': [0.404, 0.425],
    'Slight higher': [0.406, 0.430]
}

print("\nGP Predictions:")
for name, candidate in candidates_f1.items():
    pred_mean, pred_std = gp_f1.predict([candidate], return_std=True)
    print(f"  {name}: {candidate}")
    print(f"    Predicted: {pred_mean[0]:.6f} +/- {pred_std[0]:.6f}")

print("\nDECISION: [0.404, 0.425]")
print("Rationale: GP shows very flat region, test slight variation")

w9_f1_input = [0.403994, 0.424817]
print(f"\nFinal F1 input: {w9_f1_input}")

# ============================================================
# F2 (2D): NO RELIABLE MODEL - USE BEST HISTORICAL
# ============================================================

print("\n" + "="*60)
print("F2 (2D): NO RELIABLE MODEL")
print("="*60)

X_f2 = data_w8['F2']['X']
y_f2 = data_w8['F2']['y']

print("\nHistorical variance analysis:")
print("  [0.111, 0.100] -> W5: 0.1300")
print("  [0.111, 0.100] -> W7: -0.0246")
print("  Same input, 0.155 difference (119% variance)")

print("\nAttempted models:")
print("  GP with RBF kernel: Failed (noise too high)")
print("  Random Forest: Failed (noise dominates)")
print("  Bayesian Ridge: Failed (linear assumption wrong)")

print("\nSTRATEGY: Use value near W5 best")

w9_f2_input = [0.100042, 0.103196]
print(f"\nFinal F2 input: {w9_f2_input}")
print("Rationale: Near W5 [0.111, 0.100] but slightly perturbed")

# ============================================================
# F3 (3D): GAUSSIAN PROCESS WITH ARD
# ============================================================

print("\n" + "="*60)
print("F3 (3D): GAUSSIAN PROCESS WITH ARD")
print("="*60)

X_f3 = data_w8['F3']['X']
y_f3 = data_w8['F3']['y']

# Remove constraint violations
valid_mask = y_f3 > -0.5
X_f3_valid = X_f3[valid_mask]
y_f3_valid = y_f3[valid_mask]

print(f"\nTraining on valid data: {len(y_f3_valid)} points")
print(f"Output range: {y_f3_valid.min():.6f} to {y_f3_valid.max():.6f}")
print("Goal: Minimize (closer to zero)")

# GP with ARD
kernel_f3 = ConstantKernel(1.0) * Matern(
    length_scale=np.ones(3),
    length_scale_bounds=(1e-3, 10),
    nu=2.5
)

gp_f3 = GaussianProcessRegressor(
    kernel=kernel_f3,
    n_restarts_optimizer=15,
    alpha=1e-6,
    normalize_y=True
)

gp_f3.fit(X_f3_valid, y_f3_valid)

# ARD length scales
length_scales_f3 = gp_f3.kernel_.k2.length_scale
print("\nARD Length Scales:")
for i, ls in enumerate(length_scales_f3):
    importance = 1.0 / (ls + 1e-10)
    print(f"  x{i+1}: {ls:.3f} (importance: {importance:.3f})")

print("\nBest so far: W4 [0.949, 0.966, 0.808] -> -0.0786")

# Test near W4 best
candidates_f3 = {
    'W4 best': [0.949, 0.966, 0.808],
    'W8 best': [0.944, 0.965, 0.807],
    'Near W4': [0.954, 0.967, 0.808]
}

print("\nGP Predictions:")
for name, candidate in candidates_f3.items():
    pred_mean, pred_std = gp_f3.predict([candidate], return_std=True)
    print(f"  {name}: {candidate}")
    print(f"    Predicted: {pred_mean[0]:.6f} +/- {pred_std[0]:.6f}")

print("\nDECISION: [0.954416, 0.966710, 0.808259]")
print("Rationale: GP suggests slight increase in x1, x2")

w9_f3_input = [0.954416, 0.966710, 0.808259]
print(f"\nFinal F3 input: {w9_f3_input}")

# ============================================================
# F4 (4D): QUADRATIC BOWL - NEAR CENTER
# ============================================================

print("\n" + "="*60)
print("F4 (4D): QUADRATIC BOWL MODEL")
print("="*60)

X_f4 = data_w8['F4']['X']
y_f4 = data_w8['F4']['y']

print("\nBest result: W8 [0.498, 0.502, 0.500, 0.500] -> -3.9857")
print("Function: Symmetric quadratic bowl with minimum at center")

print("\nQuadratic bowl evidence:")
print("  W1 [0.5, 0.5, 0.5, 0.5] -> -3.986 (exact center)")
print("  W8 [0.498, 0.502, 0.500, 0.500] -> -3.9857")
print("  Any movement from center degrades performance")

print("\nStrategy: Test slight perturbation to verify center")

w9_f4_input = [0.488703, 0.491179, 0.485028, 0.486178]
print(f"\nFinal F4 input: {w9_f4_input}")
print("Rationale: Small move from center to confirm bowl shape")

# ============================================================
# F5 (4D): RECOVERY - REVERT x1 TO 1.0
# ============================================================

print("\n" + "="*60)
print("F5 (4D): GAUSSIAN PROCESS - RECOVERY STRATEGY")
print("="*60)

X_f5 = data_w8['F5']['X']
y_f5 = data_w8['F5']['y']

print("\nWEEK 8 DISASTER RECAP:")
print("  Week 7: [1.0, 0.853, 1.0, 0.977] -> 6158.08")
print("  Week 8: [0.855, 0.852, 1.0, 0.979] -> 4415.99")
print("  Loss: -1742 points (-28%)")
print("  Root cause: Changed x1 from 1.0 to 0.855")

# High-regime only training
high_mask = y_f5 > 1000
X_f5_high = X_f5[high_mask]
y_f5_high = y_f5[high_mask]

print(f"\nTraining on high-regime: {len(y_f5_high)} points")
print(f"Output range: {y_f5_high.min():.2f} to {y_f5_high.max():.2f}")

# GP with ARD
kernel_f5 = ConstantKernel(1.0) * Matern(
    length_scale=np.ones(4),
    length_scale_bounds=(1e-3, 10),
    nu=2.5
)

gp_f5 = GaussianProcessRegressor(
    kernel=kernel_f5,
    n_restarts_optimizer=15,
    alpha=1e-6,
    normalize_y=True
)

gp_f5.fit(X_f5_high, y_f5_high)

# ARD analysis
length_scales_f5 = gp_f5.kernel_.k2.length_scale
print("\nARD Length Scales:")
variables = ['x1', 'x2', 'x3', 'x4']
for var, ls in zip(variables, length_scales_f5):
    importance = 1.0 / (ls + 1e-10)
    status = "CRITICAL" if ls < 0.1 else "Moderate" if ls < 0.3 else "Exploratory"
    print(f"  {var}: {ls:.3f} ({status})")

print("\nREVISED INTERPRETATION:")
print("  x1 (0.041): CRITICAL - short length scale means DON'T TOUCH")
print("  x3 (0.038): CRITICAL - short length scale means DON'T TOUCH")
print("  x2 (0.165): Moderate - can explore carefully")
print("  x4 (0.312): Longest - safest to explore")

print("\nLESSON LEARNED:")
print("  Short ARD length scale = HIGH SENSITIVITY")
print("  Do NOT optimize variables with length scale < 0.1")
print("  They are boundary conditions, not tunable parameters")

print("\nRECOVERY STRATEGY:")
print("  1. REVERT x1 to 1.0 (non-negotiable)")
print("  2. Keep x3 at 1.0 (also non-negotiable)")
print("  3. Test if x2 lower OR x4 higher improves")
print("  4. SINGLE VARIABLE ISOLATION from now on")

# Test candidates
candidates_f5 = {
    'Revert to W7': [1.0, 0.853, 1.0, 0.977],
    'x2 lower': [1.0, 0.831, 1.0, 0.977],
    'x4 higher': [1.0, 0.853, 1.0, 0.988],
    'Both': [1.0, 0.831, 1.0, 0.988]
}

print("\nGP Predictions:")
for name, candidate in candidates_f5.items():
    pred_mean, pred_std = gp_f5.predict([candidate], return_std=True)
    print(f"\n{name}: {candidate}")
    print(f"  Predicted: {pred_mean[0]:.2f} +/- {pred_std[0]:.2f}")
    print(f"  vs W7 (6158): {pred_mean[0] - 6158.08:+.2f}")

print("\nDECISION: Test both x2 lower AND x4 higher")
print("Rationale: x2 and x4 have longest length scales (safest)")

w9_f5_input = [1.0, 0.830999, 1.0, 0.988351]
print(f"\nFinal F5 input: {w9_f5_input}")
print("CRITICAL: x1 reverted to 1.0, x3 stays at 1.0")

# ============================================================
# F6 (5D): GAUSSIAN PROCESS - FINE TUNING
# ============================================================

print("\n" + "="*60)
print("F6 (5D): GAUSSIAN PROCESS")
print("="*60)

X_f6 = data_w8['F6']['X']
y_f6 = data_w8['F6']['y']

print(f"\nTraining on {len(y_f6)} points")
print(f"Output range: {y_f6.min():.6f} to {y_f6.max():.6f}")
print("Goal: Minimize (less negative)")

# GP with Matern kernel
kernel_f6 = ConstantKernel(1.0) * Matern(
    length_scale=np.ones(5),
    length_scale_bounds=(1e-3, 10),
    nu=2.5
)

gp_f6 = GaussianProcessRegressor(
    kernel=kernel_f6,
    n_restarts_optimizer=15,
    alpha=1e-6,
    normalize_y=True
)

gp_f6.fit(X_f6, y_f6)

print("\nBest so far: W8 [0.258, 0.178, 0.501, 0.482, 0.412] -> -1.064")

# Test near W8 best
candidate_f6 = [0.245246, 0.162093, 0.507258, 0.481850, 0.418363]
pred_mean, pred_std = gp_f6.predict([candidate_f6], return_std=True)

print(f"\nCandidate: {candidate_f6}")
print(f"GP Prediction: {pred_mean[0]:.6f} +/- {pred_std[0]:.6f}")

print("\nDECISION: Minor refinement from W8")

w9_f6_input = [0.245246, 0.162093, 0.507258, 0.481850, 0.418363]
print(f"\nFinal F6 input: {w9_f6_input}")

# ============================================================
# F7 (6D): EMPIRICAL PATTERN - TEST BELOW PEAK
# ============================================================

print("\n" + "="*60)
print("F7 (6D): EMPIRICAL PATTERN ANALYSIS")
print("="*60)

X_f7 = data_w8['F7']['X']
y_f7 = data_w8['F7']['y']

print("\nHistorical x6 performance:")
print("  W5: x6=0.85 -> 0.836")
print("  W6: x6=0.75 -> 1.435")
print("  W7: x6=0.734 -> 1.478 (PEAK)")
print("  W8: x6=0.742 -> 1.464 (slight decline)")

print("\nPattern: Peak at x6=0.734, decline on both sides")
print("Strategy: Test x6 below 0.734 to map decline curve")

w9_f7_input = [0.022018, 0.448224, 0.257401, 0.152511, 0.397105, 0.694372]
print(f"\nFinal F7 input: {w9_f7_input}")
print("x6=0.694 (below peak, expect lower than 1.478)")

# ============================================================
# F8 (8D): BAYESIAN OPTIMIZATION - CONVERGING
# ============================================================

print("\n" + "="*60)
print("F8 (8D): BAYESIAN OPTIMIZATION")
print("="*60)

X_f8 = data_w8['F8']['X']
y_f8 = data_w8['F8']['y']

print(f"\nTraining on {len(y_f8)} points")
print("Best: W7 [0.177, 0.194, 0.170, 0.194, 0.294, 0.143, 0.109, 0.208]")
print("      -> 9.692075")

print("\nConvergence analysis:")
print("  W6: 9.676")
print("  W7: 9.692 (+0.016)")
print("  W8: 9.692 (same)")
print("Function appears to be converging")

print("\nStrategy: Small perturbations from W7 best")

w9_f8_input = [0.205880, 0.195099, 0.195658, 0.188440, 0.323931, 
               0.147753, 0.081536, 0.213205]
print(f"\nFinal F8 input: {w9_f8_input}")
print("Rationale: Minor tweaks, function likely at plateau")

# ============================================================
# WEEK 9 INPUTS SUMMARY
# ============================================================

print("\n" + "="*60)
print("WEEK 9 FINAL INPUTS SUMMARY")
print("="*60)

w9_inputs = {
    'F1': w9_f1_input,
    'F2': w9_f2_input,
    'F3': w9_f3_input,
    'F4': w9_f4_input,
    'F5': w9_f5_input,
    'F6': w9_f6_input,
    'F7': w9_f7_input,
    'F8': w9_f8_input
}

print("\nFunction | Input | Model/Strategy")
print("-" * 80)
strategies = {
    'F1': 'GP - Peak refinement',
    'F2': 'No model - Best historical',
    'F3': 'GP with ARD',
    'F4': 'Bowl model - Center test',
    'F5': 'GP with ARD - RECOVERY (x1=1.0)',
    'F6': 'GP - Fine tuning',
    'F7': 'Empirical pattern',
    'F8': 'Bayesian Opt - Converging'
}

for fn, inp in w9_inputs.items():
    print(f"{fn}      | {inp} | {strategies[fn]}")

# Portal format
print("\n" + "="*60)
print("PORTAL FORMAT (6 decimal places)")
print("="*60)

for fn, inp in w9_inputs.items():
    formatted = '-'.join([f"{v:.6f}" for v in inp])
    print(f"{fn}: {formatted}")

# ============================================================
# WEEK 9 RESULTS
# ============================================================

print("\n" + "="*60)
print("WEEK 9 ACTUAL RESULTS")
print("="*60)

w9_results = {
    'F1': 0.457432,
    'F2': 0.033044,
    'F3': -0.086019,
    'F4': -4.430389,